In [ ]:
!git clone https://github.com/peacefulmind111111/privacy-pipeline.git
%cd privacy-pipeline
%cd privacy_pipeline

In [ ]:
!pip install -r requirements.txt


In [2]:
from dataclasses import asdict
from privacy_pipeline.config import ExperimentConfig
from privacy_pipeline.training import train, train_with_outlier_clipping


In [3]:
baseline_cfg = ExperimentConfig(
    num_epochs=20,
    batch_size=512,
    lr=0.05,
    outer_momentum=0.9,
    inner_momentum=0.10,
    noise_mult=1.0,
    delta=1e-5,
    c_start=4.0,
    c_end=2.0,
    self_aug_factor=3,
    schedule_milestones=[12, 18],
    schedule_gamma=0.1,
    max_momentum_size=10000,
)
print(asdict(baseline_cfg))
train(baseline_cfg, 'outputs/baseline_run')


{'seed': 0, 'batch_size': 512, 'lr': 0.05, 'outer_momentum': 0.9, 'inner_momentum': 0.1, 'noise_mult': 1.0, 'delta': 1e-05, 'num_epochs': 20, 'c_start': 4.0, 'c_end': 2.0, 'default_clip': 1.0, 'outlier_clip': 0.5, 'high_err_threshold': 0.95, 'drop_after_frac': 0.6, 'self_aug_factor': 3, 'dataset_mean': (0.4914, 0.4822, 0.4465), 'dataset_std': (0.247, 0.2435, 0.2616), 'schedule_milestones': [12, 18], 'schedule_gamma': 0.1, 'max_momentum_size': 10000, 'device': device(type='cpu')}


100%|██████████| 170M/170M [01:23<00:00, 2.03MB/s] 
c:\Users\justi\miniconda3\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


KeyboardInterrupt: 

In [ ]:
outlier_cfg = ExperimentConfig(
    seed=0,
    batch_size=1000,
    lr=0.1,
    outer_momentum=0.9,
    inner_momentum=0.08,
    noise_mult=1.5,
    delta=1e-5,
    num_epochs=50,
    self_aug_factor=3,
    default_clip=1.0,
    outlier_clip=0.5,
    high_err_threshold=0.95,
    drop_after_frac=0.6,
    schedule_milestones=[30, 45],
    schedule_gamma=0.1,
    max_momentum_size=10000,
)
print(asdict(outlier_cfg))
train_with_outlier_clipping(outlier_cfg, 'outputs/outlier_run')


In [ ]:
# Virtual sample projection experiment
from run_experiment import run_experiment, get_default_params

vp_params = get_default_params('dp_virtual_projection')
# vp_params['num_epochs'] = 5  # example override
vp_results = run_experiment('dp_virtual_projection', params=vp_params, output='outputs/dp_virtual_projection.json')
vp_results

In [ ]:
# Local DP-SGD experiment (private + public)
from run_experiment import run_experiment, get_default_params

ld_params = get_default_params('local_dpsgd')
ld_params['EXPERIMENT'] = 'local_pub'
# ld_params['NUM_EPOCHS'] = 5  # example override
ld_results = run_experiment('local_dpsgd', params=ld_params, output='outputs/local_dpsgd.json')
ld_results